In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import open3d as o3d

from functools import partial
from pathlib import Path
from skimage import io
from skimage.feature import match_descriptors


from src.description.points.orb_descriptor import ORBDescriptor
from src.detection.points.orb import ORB
from src.keyframe_selection.every_nth_keyframe_selector import EveryNthKeyframeSelector
from src.observation.observation_creator import ObservationsCreator
from src.projection.point_projection import PointProjector
from src.geometry.transform import make_homogeneous_matrix
from src.pose_estimation.rgbd_point_pose_estimator import (
    RGBDPointPoseEstimator,
)
from src.sensor.depth import DepthImage
from src.sensor.rgb import RGBImage
from src.sensor.rgbd import RGBDImage
from src.slam.backend import G2OPrimeSLAMBackend
from src.slam.frontend import PrimeSLAMFrontend

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def get_point_cloud(points_3d):
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points_3d)

    return point_cloud


def get_line_set(lines_3d):
    lines_3d = lines_3d.reshape(-1, 2, 3)
    line_set = o3d.geometry.LineSet()
    points = []
    lines = []

    for i, edges in enumerate(lines_3d):
        points.append(edges[0])
        points.append(edges[1])
        lines.append((2 * i, 2 * i + 1))

    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.paint_uniform_color([0, 1, 0])

    return line_set

In [4]:
images_path = Path("./data/rgb")
depth_path = Path("./data/depth")
intrinsics_path = Path("./data/intrinsics.txt")

images_paths = sorted(images_path.iterdir())
depth_paths = sorted(depth_path.iterdir())
intrinsics = make_homogeneous_matrix(np.genfromtxt(intrinsics_path))
depth_scaler = 5000
images_number = len(depth_paths)

In [5]:
orb_creator = ObservationsCreator(
    ORB(nfeatures=1000),
    ORBDescriptor(),
    partial(match_descriptors, metric="hamming", max_ratio=0.8),
    PointProjector(),
    RGBDPointPoseEstimator(intrinsics, 30),
    "orb",
)

In [7]:
images = [io.imread(path) for path in images_paths]
depths = [io.imread(path) for path in depth_paths]

frames = [
    RGBDImage(RGBImage(img), DepthImage(depth, intrinsics, depth_scaler))
    for img, depth in zip(images, depths)
]
keyframe_selector = EveryNthKeyframeSelector(n=1)

slam = PrimeSLAMFrontend(
    [orb_creator],
    keyframe_selector,
    init_pose=np.eye(4),
)

for frame in frames:
    slam.process_frame(frame)

projector = PointProjector()
backend = G2OPrimeSLAMBackend(intrinsics)
optimized_poses, optimized_points = backend.optimize(slam.graph)

o3d.visualization.draw_geometries(
    [
        get_point_cloud(np.array(optimized_points)).paint_uniform_color(
            (0, 1, 0)
        ),
    ]
)